<a href="https://colab.research.google.com/github/napaphach/credit-scoring/blob/main/Copy_of_Datascience_Credit_Scoring__test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder, OneHotEncoder
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from sklearn.metrics import classification_report, mean_squared_error, plot_confusion_matrix, plot_precision_recall_curve
from scipy.stats import randint
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix



In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/napaphach/credit-scoring/main/credit-scoring.csv")

In [ ]:
df_rename = data.rename(columns ={"SeriousDlqin2yrs": "Default", 
                     "RevolvingUtilizationOfUnsecuredLines":"Rev",
                     "NumberOfTime30-59DaysPastDueNotWorse":"Pastdue1",
                     "NumberOfOpenCreditLinesAndLoans":"CreditLine",
                     "NumberOfTimes90DaysLate":"Pastdue2",
                     "NumberRealEstateLoansOrLines":"RealEstate",
                     "NumberOfTime60-89DaysPastDueNotWorse":"Pastdue3",
                     "NumberOfDependents" : "Dependents"})
df = df_rename.drop (columns=['Unnamed: 0']) 

In [ ]:
df['Default'].value_counts()

0    139974
1     10026
Name: Default, dtype: int64

In [ ]:
# Visualize imbalance dataset - imbalance class
sns.set(font_scale=1.4)
df['Default'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
plt.xlabel("Default", labelpad=14)
plt.ylabel("Number of observations", labelpad=14)
plt.title("Person experienced 90 days past due delinquency or worse", y=1.02);

In [ ]:
#Drop missing value of all variables
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["age"], inplace=True)
df.dropna(subset = ["MonthlyIncome"], inplace=True)

In [ ]:
df

In [ ]:
#Drop error value of variable age and monthly income
df = df[df.age != 0]
df = df[df.MonthlyIncome != 0]

In [ ]:
df.describe(include="all")
x = df.drop (["Default"], axis = 1)
y = df["Default"]
# Drop column Unnamed:0 as it isn't important informationtion will be used in model 

In [ ]:
x

In [ ]:
y

In [ ]:
#Visualize correlation between variables
plt.figure(figsize=(20,15))
sns.heatmap(df.corr(), annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
# Spliting train and test set before modeling data
# Make sure the distribution of class in the test set and traning set are the same using stratify=y
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, shuffle=True, random_state=11, stratify=y)

In [ ]:
pipeline = make_pipeline(
    make_column_transformer(
        (make_pipeline(
            MinMaxScaler(),
        ), ["Rev", "DebtRatio"]),
    remainder="passthrough"),
    RandomUnderSampler(),
    RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=-1,
                                        oob_score=False, random_state=1909,
                                        verbose=0, warm_start=False)
)

In [ ]:
# Evaluate model without hyperparametter tuning/ parameter optimization
cv = cross_validate(pipeline,x_train,y_train, cv=10, scoring= ["f1"], n_jobs=-1, return_train_score=True)

In [ ]:
cv

{'fit_time': array([2.8749733 , 2.8587122 , 2.85625291, 2.77412295, 2.75338197,
        2.74961185, 2.85400414, 2.86703587, 2.66350198, 2.7065568 ]),
 'score_time': array([0.31189775, 0.32862878, 0.31737089, 0.31066847, 0.23200846,
        0.33499813, 0.31182933, 0.32721424, 0.34031177, 0.34105849]),
 'test_f1': array([0.31771894, 0.33105335, 0.31983668, 0.32521441, 0.31840448,
        0.32515337, 0.32753825, 0.32689655, 0.32022084, 0.32258065]),
 'train_f1': array([0.42219624, 0.42106847, 0.42385912, 0.4253459 , 0.42736545,
        0.42233541, 0.41600059, 0.42672463, 0.4224714 , 0.42885065])}

In [ ]:
#These are only example of hyperparameter used in the model 
#Create distribution to indentifined what model parameter that we want to optimize
# All pipeline or only some part of pipeline can be optimize
distribution = {
    "RandomForestClassifier__ n_estimators":randint(low=200, high=400) ,
    #indentified number of tree using randint distribution
    "RandomForestClassifier__ max_depth":randint(low=75, high=125) ,
    #How deep the tree should be
    "RandomForestClassifier__ max_features": ["auto", "sqrt", "log2"]
}

In [ ]:
Random_model = RandomizedSearchCV(pipeline, distribution, n_iter=10, scoring = ["f1"], n_jobs =-1,
                           random_state=11, cv=5,return_train_score = True, refit=False
                           )

In [ ]:
Random_model.fit(x_train, y_train)

ValueError: ignored

In [ ]:
Random_model.predict(x_test)

NotFittedError: ignored

In [ ]:
x_train

In [ ]:
cv = cross_validate(estimator=pipeline, X=x, y=y, scoring=["f1", "recall", "precision", "accuracy"], cv=10, n_jobs=-1, return_train_score=True, verbose=1)

In [ ]:
cv

In [ ]:
#checking the distribution
(y_test == 0).sum()

27586

In [ ]:
#checking the distribution
(y_test == 1).sum()

2073